# Using a Calculator Tool with Claude
In this recipe, we'll demonstrate how to provide Claude with a simple calculator tool that it can use to perform arithmetic operations based on user input. We'll define the calculator tool and show how Claude can interact with it to solve mathematical problems.

This recipe is a Ruby adaptation of the [original Python recipe](https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/calculator_tool.ipynb).

## Step 1: Set up the environment
First, let's install the `anthropic-rb` gem.

In [42]:
require "bundler/inline"

gemfile do
  source "https://rubygems.org"

  gem "anthropic-rb"
end

Then, we'll initialize the library with our API key.

In [43]:
require 'anthropic'

Anthropic.setup do |config|
  config.api_key = ENV.fetch('ANTHROPIC_API_KEY')
end

{:completions=>[{"version"=>"2023-06-01", "endpoint"=>"/v1/complete", "schema"=>{"type"=>"object", "required"=>["model", "prompt", "max_tokens_to_sample"], "properties"=>{"model"=>{"type"=>"string"}, "prompt"=>{"type"=>"string"}, "max_tokens_to_sample"=>{"type"=>"integer"}, "stop_sequences"=>{"type"=>"array", "items"=>{"type"=>"string"}}, "temperature"=>{"type"=>"number"}, "top_k"=>{"type"=>"integer"}, "top_p"=>{"type"=>"number"}, "metadata"=>{"type"=>"object"}, "stream"=>{"type"=>"boolean"}}, "additionalProperties"=>false}}], :messages=>[{"version"=>"2023-06-01", "endpoint"=>"/v1/messages", "schema"=>{"type"=>"object", "required"=>["model", "messages", "max_tokens"], "properties"=>{"model"=>{"type"=>"string"}, "messages"=>{"type"=>"array"}, "max_tokens"=>{"type"=>"integer"}, "system"=>{"type"=>"string"}, "stop_sequences"=>{"type"=>"array", "items"=>{"type"=>"string"}}, "temperature"=>{"type"=>"number"}, "top_k"=>{"type"=>"integer"}, "top_p"=>{"type"=>"number"}, "metadata"=>{"type"=>"o

## Step 2: Define the calculator tool
We'll define a simple calculator tool that can perform basic arithmetic operations. The tool will take a mathematical expression as input and return the result.

Note that we are calling eval on the outputted expression. This is bad practice and should not be used generally but we are doing it for the purpose of demonstration.

In [44]:
def calculate(expression)
  # Remove any non-digit or non-operator characters from the expression
  expression = expression.gsub(/[^0-9+\-*\/().]/i, '')
  
  begin
    # Evaluate the expression using the built-in eval() function
    result = eval(expression)
    result.to_s
  rescue SyntaxError, ZeroDivisionError, NameError, TypeError, FloatDomainError
    "Error: Invalid expression"
  end
end

:calculate

We then define the calculator tool with an input schema that expects a single expression property of type string.

In [45]:
@tools = [
    {
        "name": "calculator",
        "description": "A simple calculator that performs basic arithmetic operations.",
        "input_schema": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "The mathematical expression to evaluate (e.g., '2 + 3 * 4')."
                }
            },
            "required": ["expression"]
        }
    }
]

[{:name=>"calculator", :description=>"A simple calculator that performs basic arithmetic operations.", :input_schema=>{:type=>"object", :properties=>{:expression=>{:type=>"string", :description=>"The mathematical expression to evaluate (e.g., '2 + 3 * 4')."}}, :required=>["expression"]}}]

## Step 3: Interact with Claude
Now, let's see how Claude can interact with the calculator tool to solve mathematical problems.

In [46]:
def chat_with_claude(user_message)
  puts("#{'=' * 50}\nUser Message: #{user_message}\n#{'=' * 50}")
  message = Anthropic.messages(beta: 'tools-2024-04-04').create(
    model: 'claude-3-opus-20240229',
    max_tokens: 200,
    tools: @tools,
    messages: [{ role: 'user', content: user_message }]
  )
  puts("\nInitial Response:")
  puts("Stop Reason: #{message[:stop_reason]}")
  puts("Content: #{message[:content]}")

  if message[:stop_reason] == 'tool_use'
    tool_use_block = message[:content].find { |block| block[:type] == 'tool_use' }
    puts("\nTool Used: #{tool_use_block[:name]}")
    puts("Tool Input: #{tool_use_block[:input]}")
    tool_result = if tool_use_block[:name] == 'calculator'
                    calculate(tool_use_block[:input][:expression])
                  else
                    nil
                  end
      
    puts("Tool Result: #{tool_result}")
    response = Anthropic.messages(beta: 'tools-2024-04-04').create(
      model: 'claude-3-opus-20240229',
      max_tokens: 200,
      tools: @tools,
      messages: [
        { role: 'user', content: user_message },
        { role: 'assistant', content: message[:content] },
        {
          role: 'user',
          content: [
            {
              type: 'tool_result',
              tool_use_id: tool_use_block[:id],
              content: tool_result
            }
          ]
        }
      ]
    )
    puts("\nFinal Response: #{response[:content]}\n")
  end
end
  

:chat_with_claude

## Step 4: Try it out!
Let's try giving Claude a few example math questions now that it has access to a calculator.

In [47]:
chat_with_claude("What is the result of 1,984,135 * 9,343,116?")
chat_with_claude("Calculate (12851 - 593) * 301 + 76")
chat_with_claude("What is 15910385 divided by 193053?")

User Message: What is the result of 1,984,135 * 9,343,116?

Initial Response:
Stop Reason: tool_use
Content: [{:type=>"text", :text=>"<thinking>\nTo answer this question, I will need to use the calculator tool.\nThe calculator tool takes a parameter \"expression\" which represents the mathematical expression to evaluate. \nThe user has provided the mathematical expression \"1,984,135 * 9,343,116\" directly in their query.\nSo I have the necessary parameter value to call the calculator tool.\n</thinking>"}, {:type=>"tool_use", :id=>"toolu_01JzKqaDpCNZBGvMNaBCQ2hT", :name=>"calculator", :input=>{:expression=>"1984135 * 9343116"}}]

Tool Used: calculator
Tool Input: {:expression=>"1984135 * 9343116"}
Tool Result: 18538003464660

Final Response: [{:type=>"text", :text=>"The result of multiplying 1,984,135 by 9,343,116 is 18,538,003,464,660."}]
User Message: Calculate (12851 - 593) * 301 + 76

Initial Response:
Stop Reason: tool_use
Content: [{:type=>"text", :text=>"<thinking>\nTo answer th